### Import packages

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import torch.nn.functional as F

## Dataset And Dataloader

In [3]:
class TemporalDataset(Dataset):
    def __init__(self, features, labels, sequence_length):
        self.features = features
        self.labels = labels
        self.sequence_length = sequence_length
        self.max_length = len(features)

    def __len__(self):
        return len(self.features) - self.sequence_length + 1

    def __getitem__(self, idx):
        idx_end = idx + self.sequence_length

        # Extract the sequence of features
        sequence_features = self.features[idx:idx_end]

        # Handle NaN values: replace NaNs with zeros or any other strategy
        sequence_features = torch.tensor(sequence_features, dtype=torch.float32)
        sequence_features[torch.isnan(sequence_features)] = 0.0

        # Extract the label
        label = torch.tensor(self.labels[idx_end - 1], dtype=torch.long)

        return sequence_features, label


# File paths for training, validation, and testing
train_file = "D:/iMobie/Final Project/final test of LEGO CAR assambly/DATA OF ONLY RIGHT HAND DATA/concatenated_data_one_hot_train.csv"
test_file  = "D:/iMobie/Final Project/final test of LEGO CAR assambly/DATA OF ONLY RIGHT HAND DATA/concatenated_data_one_hot_test.csv"
val_file = "D:/iMobie/Final Project/final test of LEGO CAR assambly/DATA OF ONLY RIGHT HAND DATA/concatenated_data_one_hot_val.csv"

# Sequence length for creating sequences
sequence_length = 30 # You can adjust this based on your preference

# Read CSV files
train_data = pd.read_csv(train_file)
val_data = pd.read_csv(val_file)
test_data = pd.read_csv(test_file)

# Specify columns to normalize
columns_to_normalize = ['RotationRate',	'HorizontalRate',	'VerticalRate'	,'DuctionRate'	,'FlexionRate',	'StrengthAmplitude']  # Replace with your actual column names

# Normalize selected columns using StandardScaler
scaler = StandardScaler()
train_data[columns_to_normalize] = scaler.fit_transform(train_data[columns_to_normalize])
val_data[columns_to_normalize] = scaler.transform(val_data[columns_to_normalize])
test_data[columns_to_normalize] = scaler.transform(test_data[columns_to_normalize])

# Extract features and labels
train_features = train_data.drop(['temporal_order', 'action'], axis=1).values
train_labels = train_data['action'].values

val_features = val_data.drop(['temporal_order', 'action'], axis=1).values
val_labels = val_data['action'].values

test_features = test_data.drop(['temporal_order', 'action'], axis=1).values
test_labels = test_data['action'].values

# Use LabelEncoder to convert string labels to numerical labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)
test_labels = label_encoder.transform(test_labels)

# Create datasets
train_dataset = TemporalDataset(train_features, train_labels, sequence_length)
val_dataset = TemporalDataset(val_features, val_labels, sequence_length)
test_dataset = TemporalDataset(test_features, test_labels, sequence_length)

# Batch size for the data loader
batch_size = 32

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print a sample from the training DataLoader
for batch_idx, (inputs, labels) in enumerate(train_loader):
    # Print the content of the first batch
    if batch_idx == 0:
        print(f"Batch {batch_idx + 1} - Inputs shape: {inputs.shape}, Labels shape: {labels.shape}")
        for i in range(min(5, batch_size)):  # Print only the first 5 samples
            print(f"Sample {i + 1}: {inputs[i]}, Label: {labels[i]}")

    # Break out of the loop after printing the first batch
    break


Batch 1 - Inputs shape: torch.Size([32, 30, 42]), Labels shape: torch.Size([32])
Sample 1: tensor([[ 1.6648e+01,  1.5082e-01, -1.5210e-02,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.6678e+01,  4.0196e-01,  3.4363e-01,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.6718e+01,  2.4442e-01,  2.5817e-01,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 1.7447e+01, -1.6774e+00,  1.3619e-02,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.7479e+01, -1.6774e+00, -9.0036e-02,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.7501e+01, -1.6774e+00, -9.0036e-02,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]]), Label: 3
Sample 2: tensor([[23.7017,  0.7714,  0.2793,  ...,  0.0000,  0.0000,  0.0000],
        [23.7309,  0.6564,  0.4872,  ...,  0.0000,  0.0000,  0.0000],
        [23.7608,  0.7688,  0.5276,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [24.5020,  0.6502,  0.1624

## LSTM Model

In [4]:
# Define the LSTM model with a softmax layer
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Use the output from the last time step
        out = self.softmax(out)  # Apply softmax
        return out
    
    
# Set the input size, hidden size, number of layers, and output size
input_size = train_features.shape[1]
hidden_size = 128  # Increase the hidden size
num_layers = 2
output_size = len(label_encoder.classes_)

# Instantiate the model
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of training epochs
num_epochs = 400

# Lists to store training and validation losses
train_losses = []
val_losses = []



# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_train_losses = []
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_train_losses.append(loss.item())

        if batch_idx % 100 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item()}")

    # Calculate and store the average training loss for the epoch
    avg_train_loss = np.mean(epoch_train_losses)
    train_losses.append(avg_train_loss)

    # Validation loop
    model.eval()
    with torch.no_grad():
        epoch_val_losses = []
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            epoch_val_losses.append(val_loss.item())

        # Calculate and store the average validation loss for the epoch
        avg_val_loss = np.mean(epoch_val_losses)
        val_losses.append(avg_val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss}")

# Save the trained model and label encoder
torch.save(model.state_dict(), './model.pth')
torch.save(label_encoder.classes_, '.encoder.pth')



# Plot training and validation losses
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Losses Over Time')
plt.legend()
plt.show()

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []

    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Collect predictions and labels for confusion matrix
        all_predictions.extend(predicted.numpy())
        all_labels.extend(labels.numpy())

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Confusion Matrix for the test set
conf_matrix_test = confusion_matrix(all_labels, all_predictions)

# Plot confusion matrix for the test set
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_test, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title("Confusion Matrix - Test Set")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

Epoch 1/400, Batch 1/4457, Loss: 1.9431930780410767
Epoch 1/400, Batch 101/4457, Loss: 1.748540997505188
Epoch 1/400, Batch 201/4457, Loss: 1.5659160614013672
Epoch 1/400, Batch 301/4457, Loss: 1.6819970607757568
Epoch 1/400, Batch 401/4457, Loss: 1.3945823907852173
Epoch 1/400, Batch 501/4457, Loss: 1.500264286994934
Epoch 1/400, Batch 601/4457, Loss: 1.4466806650161743
Epoch 1/400, Batch 701/4457, Loss: 1.4145846366882324
Epoch 1/400, Batch 801/4457, Loss: 1.3798861503601074
Epoch 1/400, Batch 901/4457, Loss: 1.5079249143600464
Epoch 1/400, Batch 1001/4457, Loss: 1.429541826248169
Epoch 1/400, Batch 1101/4457, Loss: 1.5117307901382446
Epoch 1/400, Batch 1201/4457, Loss: 1.4456894397735596
Epoch 1/400, Batch 1301/4457, Loss: 1.4755032062530518
Epoch 1/400, Batch 1401/4457, Loss: 1.5555425882339478
Epoch 1/400, Batch 1501/4457, Loss: 1.467800259590149
Epoch 1/400, Batch 1601/4457, Loss: 1.388073444366455
Epoch 1/400, Batch 1701/4457, Loss: 1.5655490159988403
Epoch 1/400, Batch 1801/445

KeyboardInterrupt: 